In [8]:
import os
import torch
from models.resnet_balance import resnet_fedbalance
import sys
sys.path.append('/mnt/data/th')

root_path = "/mnt/data/th/FedTH/data"
datadir = os.path.join(root_path, "dataset", "cifar10")

from FedML.fedml_api.data_preprocessing.cifar10.data_loader import get_client_idxes_dict, get_client_dataloader

import numpy as np


In [9]:
dict_client_idexes, class_num, client_infos = get_client_idxes_dict(
    datadir, "hetero", 0.2, 100)
traindl,testdl = get_client_dataloader(
    datadir, 64, dict_client_idexes, client_idx=0, train=True)


INFO:root:*********partition data***************
INFO:root:N = 50000
INFO:root:traindata_cls_counts = {0: {0: 1, 1: 7, 2: 45, 5: 120, 6: 3, 7: 81, 8: 1, 9: 1}, 1: {0: 38, 1: 67, 2: 13, 4: 11, 5: 28, 6: 24, 9: 1}, 2: {0: 43, 2: 7, 5: 81, 7: 180, 8: 9, 9: 14}, 3: {1: 29, 4: 4, 6: 4, 7: 1, 8: 126, 9: 180}, 4: {0: 245, 1: 6, 2: 4, 4: 74, 5: 32, 6: 47, 7: 22, 8: 309}, 5: {4: 1, 6: 8, 7: 112, 8: 5, 9: 1}, 6: {0: 4, 2: 5, 3: 9, 4: 870}, 7: {0: 27, 1: 142, 3: 106, 4: 4, 5: 3, 6: 44, 7: 180}, 8: {0: 473, 1: 14, 2: 71}, 9: {1: 10, 2: 1, 3: 5, 4: 5, 5: 2, 6: 33, 7: 64, 8: 3, 9: 665}, 10: {0: 355, 1: 366}, 11: {2: 283, 3: 2, 7: 28, 8: 5, 9: 204}, 12: {0: 11, 2: 65, 3: 1, 4: 103, 5: 58, 7: 3, 8: 694}, 13: {0: 2, 1: 93, 2: 234, 3: 21, 5: 7, 7: 23, 8: 87, 9: 171}, 14: {0: 36, 1: 20, 3: 2, 5: 409, 7: 55}, 15: {1: 89, 2: 1, 3: 106, 4: 364}, 16: {1: 1, 2: 14, 4: 1, 5: 68, 6: 69, 7: 5, 8: 144, 9: 166}, 17: {5: 437, 6: 9, 8: 7, 9: 1}, 18: {0: 30, 2: 2, 3: 3, 4: 18, 5: 97, 8: 2, 9: 34}, 19: {1: 1, 3: 209, 

In [10]:
client_cnts = {}
# print(self.client_infos)

for client,info in client_infos.items():
    cnts = []
    for c in range(10):
        if c in info.keys():
            num = info[c]
        else:
            num = 0
        cnts.append(num)

    cnts = torch.FloatTensor(np.array(cnts))
    client_cnts.update({client:cnts})

In [11]:
client_dis = client_cnts[0]

dist = client_dis / client_dis.sum() #个数的比例
cdist = dist / dist.max()
# cdist = cdist * (1.0 - self.args.alpha) + self.args.alpha
cdist = cdist.reshape((1, -1))

In [12]:
print(dist,cdist,cdist.shape)

tensor([0.0039, 0.0270, 0.1737, 0.0000, 0.0000, 0.4633, 0.0116, 0.3127, 0.0039,
        0.0039]) tensor([[0.0083, 0.0583, 0.3750, 0.0000, 0.0000, 1.0000, 0.0250, 0.6750, 0.0083,
         0.0083]]) torch.Size([1, 10])


In [15]:
device = torch.device("cuda")

model = resnet_fedbalance().to(device)


for epoch in range(2):

    for batch_idx, (images, labels) in enumerate(traindl):
        # logging.info(images.shape)
        images, labels = images.to(device), labels.to(device)
        log_probs = model(images,cdist.to(device))
